### Reading and Processing a single file from 'song_data' and 'log_data' then loading the data into the tables. This notebook contains detailed instructions on the ETL process for each of the tables.**

In [1]:
import psycopg2 
import pandas as pd
import json
import os
from SQL_Queries import *
import glob2


### Create  Database 

In [2]:
import Create_Tables as ct

ct.main()

database "sparkify" is being accessed by other users
DETAIL:  There is 1 other session using the database.

database "sparkify" already exists



### Connect to Databaes 

In [3]:
import Create_Tables as ct

ct.main()

try:
    conn = psycopg2.connect(user = "postgres",
                           password = "root",
                           port = "5432",
                           host = "127.0.0.1",
                           database = "sparkify")
except psycopg2.Erorr as e:
    print("Error while connecting to database ")
    print(e)
    
#get cursor 
cur  = conn.cursor()

#set session to auto commit
conn.set_session(autocommit = True)

database "sparkify" is being accessed by other users
DETAIL:  There is 1 other session using the database.

database "sparkify" already exists



In [4]:
#method to get files from directory
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob2.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [8]:
all_files = []
for root , dirs , files in os.walk("/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/"):
    files  = glob2.glob(os.path.join(root , '*.json'))
    for file  in files:
        all_files.append(os.path.abspath(file))
        
    print(all_files)

[]
[]
['/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-19-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-13-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-06-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-07-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-02-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-22-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-25-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with Post

In [7]:
log_data_path = "/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/" #2018/11/2018-11-01-events.json"
files = get_files(log_data_path)
print(files)
log_data_df = []
for file in files:
    
    log_data  = pd.read_json(file , lines= True)
    log_data.head()
    log_data_df.append(log_data)
        
log_data_df = pd.DataFrame(log_data_df)
#log_data_df.head()

['/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-19-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-13-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-06-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-07-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-02-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-22-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-25-events.json', '/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL

/home/bebo/anaconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


#  Processe  `log_data`
here will preform ETL on **log data** dataset, to create `time` and `users` dimentional tables.



we will preform ETL on a single log file

In [5]:
#path of JSON files
log_data_file_path = "/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/log_data/2018/11/2018-11-01-events.json"

#loading loag data from json to pandas DataFrame
log_Data = pd.read_json(log_data_file_path, lines=True)

log_Data

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #1: Time Table
#### Extracting Data for Time Table
- Filtering the records by `NextSong` action
- Converting the `ts` timestamp column to datetime
- Extracting the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and setting `time_data` to a list 
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specifying labels for these columns and set to `column_labels`
- Creating a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [6]:
#Filtering records by NextSong action
log_Data = log_Data[log_Data.page == 'NextSong']
log_Data

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1540266185796,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [7]:
#Convert the ts timestamp column to datetime
time = pd.to_datetime(log_Data['ts'], unit='ms')
time

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [8]:
#Extracting the timestamp, hour, day, week of year, month, year, and weekday from the ts
time_data = [time, time.dt.hour , time.dt.day, time.dt.isocalendar().week , time.dt.month , time.dt.year, time.dt.weekday ]
column_labels = ['start_time', 'hour' , 'day', 'week', 'month', 'year', 'weekday']


In [9]:
#combining `column_labels` and `time_data` into a dictionary
time_dict = dict(zip(  column_labels, time_data))

#Create a dataframe, `time_df,` containing the time data for this file
time_df = pd.DataFrame(time_dict)
                
time_df.head()

,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Inserting Records into Time Table

In [10]:
# iterating over rows using iterrows() function 
for index , row in time_df.iterrows():
    print(row)
    #inser into time table
    try:
        cur.execute(time_table_insert , list(row))
        conn.commit()
    except psycopg2.Error as e:
        print("Error while insert into time table")
        print(e)

start_time    2018-11-01 21:01:46.796000
hour                                  21
day                                    1
week                                  44
month                                 11
year                                2018
weekday                                3
Name: 2, dtype: object
start_time    2018-11-01 21:05:52.796000
hour                                  21
day                                    1
week                                  44
month                                 11
year                                2018
weekday                                3
Name: 4, dtype: object
start_time    2018-11-01 21:08:16.796000
hour                                  21
day                                    1
week                                  44
month                                 11
year                                2018
weekday                                3
Name: 5, dtype: object
start_time    2018-11-01 21:11:13.796000
hour                         

In [11]:
#test if you've successfully added records to time table.
try:
    cur.execute("SELECT * FROM time;")
    
except psycopg2.Error as e:
        print(e)
        
rows = cur.fetchall()
for i in rows :
    print(i)

(datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 8, 16, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 11, 13, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 17, 33, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 24, 53, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 28, 54, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 42, 0, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 52, 5, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 21, 55, 25, 796000), 21, 1, 44, 11, 2018, '3')
(datetime.datetime(2018, 11, 1, 22, 23, 14, 796000), 22, 1, 44, 11, 2018, '3')


## 2#: users Table
#### Extracting Data for Users Table
- Selecting user ID, first name, last name, gender and level to `user_df`

In [12]:
users_df = log_Data[['userId', 'firstName', 'lastName', 'gender', 'level']]
users_df = users_df.drop_duplicates().dropna()
users_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free


#### Inserting Records into Users Table

In [13]:
# iterating over rows using iterrows() function 
for index , row in users_df.iterrows():
    
    #inser into users table
    try:
        cur.execute(user_table_insert , list(row))
        conn.commit()
    except psycopg2.Error as e:
        print("Error while insert into user table")
        print(e)

In [14]:
#test if you've successfully added records to users table.
try:
    cur.execute("SELECT * FROM users;")
    conn.commit()
except psycopg2.Error as e:
        print(e)
        
rows = cur.fetchall()
for i in rows :
    print(i)

(8, 'Kaylee', 'Summers', 'F', 'free')
(10, 'Sylvie', 'Cruz', 'F', 'free')
(26, 'Ryan', 'Smith', 'M', 'free')
(101, 'Jayden', 'Fox', 'M', 'free')


# Process `song_data`
In this first part, we'll perform ETL on the second dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.

In [15]:
#open files 
files  = get_files("/home/bebo/Data Engineering Course Udcity/Project 1 Data Modeling with PostgreSQL /data/song_data/") 
song_data_df  = pd.read_json(files[0] , lines  = True)
song_data_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


## #3: `songs` Table
#### Extracting Data for Songs Table
- Selecting song ID, title, artist ID, year, and duration

In [16]:
song_df = song_data_df[['song_id', 'title', 'artist_id','year', 'duration'] ]
song_df.drop_duplicates().dropna()
song_df.head()

,song_id,title,artist_id,year,duration
0,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546


In [17]:
song_values = song_df.values
song_first_record = song_values[0]
song_data = song_first_record.tolist()
song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

In [18]:
try:
    cur.execute(song_table_insert , song_data)
except psycopg2.Error as e:
    print(e)
conn.commit()

insert or update on table "songs" violates foreign key constraint "songs_artist_id_fkey"
DETAIL:  Key (artist_id)=(ARMJAGH1187FB546F3) is not present in table "artists".



## #4: `artists` Table
#### Extracting Data for Artists Table
- Selecting artist ID, name, location, latitude, and longitude

In [19]:
artist_data_df = song_data_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data_values = artist_data_df.values
artist_first_record = artist_data_values[0]
artist_data = artist_first_record.tolist()
artist_data

['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892]

In [20]:
try:
    cur.execute(artist_table_insert , artist_data)
except psycopg2.Error as e:
    print(e)
conn.commit()

## #5: `songplays` Table
#### Extracting Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, we'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implementing the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Selecting the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Inserting Records into Songplays Table

In [23]:
for index , row in log_Data.iterrows():
    cur.execute(song_select , (row.song, row.artist, row.length))
    result = cur.fetchone()
    if result:
        songId , artistId  = result
    else :
        songId , artistId  = None , None 
    
    #insert in to songplays 
    songplays_row = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level ,  songId , artistId , row.sessionId , row.location , row.userAgent)
    cur.execute(songplays_table_insert , songplays_row)
    conn.commit()

In [24]:
conn.close()